## Document Similarity

Using NLTK and GenSim, we will use TF-IDF and LSI to find document similarities quickly and accurately. Let's begin by copying our preprocessing pipeline and running it on the Brown corpus.

In [1]:
from nltk import word_tokenize
from nltk.corpus import brown, stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag


sw = stopwords.words("english")
sw += ""

lemmatizer = WordNetLemmatizer()

# only use Noun, Verb, Adjective, and Adverb. Default everything else to Noun
wordnet_map = {"N": wordnet.NOUN, "V": wordnet.VERB, "J": wordnet.ADJ, "R": wordnet.ADV}


corpus = brown

def remove_punc(doc):
    sentences = []
    for sentence in doc:
        sentence = " ".join(sentence)  # converts from list of words to a single string
        words = word_tokenize(sentence)  # separates punctuation
        
        # removes punctuation, makes lowercase
        words= [word.lower() for word in words if word.isalnum()]
        sentences.append(words)
    return sentences


def remove_sw(doc):
    sentences = []
    for sentence in doc:
        sentence = [word for word in sentence if word not in sw]
        sentences.append(sentence)
    return sentences


def lemmatize(doc):
    sentences = []
    for sentence in doc:
        tagged = pos_tag(sentence)
        sentence = [
            lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN))
            for word, pos in tagged
        ]
        sentences.append(sentence)
    return sentences

def preprocess(doc):
    doc = remove_punc(doc)
    doc = remove_sw(doc)
    doc = lemmatize(doc)
    return doc

def preprocess_corpus(corpus):
    processed = []
    for category in corpus.categories():
        print(category)
        doc = corpus.sents(categories=category)
        processed.append(preprocess(doc))
    return processed

# can take a minute or two
corpus = preprocess_corpus(corpus)

adventure
belles_lettres
editorial
fiction
government
hobbies
humor
learned
lore
mystery
news
religion
reviews
romance
science_fiction


A quick look at the output.

In [2]:
# combines sentences in each document
corpus = [" ".join([" ".join(sent) for sent in doc]).split(" ") for doc in corpus]

# a sample of the first 100 processed words from the adventure document
print(corpus[0][0:100])

['dan', 'morgan', 'tell', 'would', 'forget', 'ann', 'turner', 'well', 'rid', 'certainly', 'want', 'wife', 'fickle', 'ann', 'married', 'ask', 'trouble', 'rationalization', 'sometimes', 'wake', 'middle', 'night', 'think', 'ann', 'could', 'get', 'back', 'sleep', 'plan', 'dream', 'revolve', 'around', 'much', 'long', 'felt', 'nothing', 'easy', 'thing', 'would', 'sell', 'al', 'budd', 'leave', 'country', 'stubborn', 'streak', 'would', 'allow', 'best', 'antidote', 'bitterness', 'disappointment', 'poison', 'hard', 'work', 'find', 'tire', 'enough', 'night', 'go', 'sleep', 'simply', 'exhausted', 'stay', 'awake', 'day', 'find', 'think', 'less', 'often', 'ann', 'day', 'hurt', 'little', 'dull', 'little', 'less', 'poignant', 'plenty', 'work', 'summer', 'unusually', 'dry', 'hot', 'spring', 'produce', 'small', 'stream', 'ordinary', 'year', 'grass', 'meadow', 'come', 'fast', 'warm', 'weather', 'could', 'afford', 'lose', 'drop']


### Dictionary

While we used NLTK for pre-processing, we will use GenSim for Document Similarity. GenSim was designed specifically for this use case, and has many helpful features. One such tool is the Dictionary. We can pass to the dictionary a list of documents, each containing a list of words. The dictionary will keep track of the unique tokens and their frequency in both the corpus and individual documents.

In [3]:
from gensim import corpora, models, similarities

dictionary = corpora.Dictionary(corpus)
dictionary.filter_n_most_frequent(1)  # removes ""
num_words = len(dictionary)
print(dictionary)
print()

print("Most Frequent Words")
top10 = sorted(dictionary.cfs.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (id, freq) in enumerate(top10):
    print(i, freq, dictionary[id])

Dictionary(33663 unique tokens: ['1', '10', '125', '15th', '16']...)

Most Frequent Words
0 3473 one
1 2843 would
2 2778 say
3 2327 make
4 1916 time
5 1816 go
6 1777 could
7 1665 new
8 1659 year
9 1575 take


As you can see, our corpus contains 33,663 unique tokens, and some of the most frequent words include "one", "would", and "say".

### Bag of Words

To create document similarities, we must create mathematical representations for each document. A vector is a list of numbers describing an object. As a first step, we can give each unique token in the dictionary a key. Then for each document, the value of the vector at that key is the number of occurances of that specific token. This method is called bag of words.

Example:

['to', 'be', 'or', 'not', 'to', 'be']


{0: 'to', 1: 'be', 2: 'or', 3: 'not'}


[2, 2, 1, 1]

GenSim makes it exceedingly easy to run bag of words on a corpus

In [4]:
corpus_bow = [dictionary.doc2bow(doc) for doc in corpus]
print(len(corpus_bow[0]))
print(corpus_bow[0][:20])

6106
[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 2), (13, 2), (14, 2), (15, 1), (16, 2), (17, 2), (18, 3), (19, 1)]


After running bag of words on the entire corpus, we look at the bag of words vector for the first document. It has 6106 unique tokens. Printing the first 20, we see most only appear once, while some appear two or three times. 

### Term Frequency - Inverse Document Frequency (TF-IDF)

Term Frequency - Inverse Document Frequency is a popular tool to improve the usefullness of a bag of words representation. Instead of solely counting the number of occurances, we make two modifications. First, we look at the term frequency instead of term count. This helps normalize against document length. Second, we weigh each term by how many documents the term appears in. If a common word like 'new' or 'make' appears, it means less than if a rare word like 'whale' appears. When evaluating document similarity, two documents are more similar if they share the word 'whale' than if they share the word 'new'.

Once again, we can implement this in just a few lines with GenSim

In [5]:
tfidf_model = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf_model[corpus_bow]
print(len(corpus_tfidf[0]))
print(corpus_tfidf[0][:20])

5575
[(0, 0.001040495879718581), (1, 0.0011016669638018743), (2, 0.002351365659027428), (3, 0.002351365659027428), (4, 0.0013108697793088472), (5, 0.005170600993729588), (6, 0.003391861538746009), (7, 0.004130105114011007), (8, 0.003391861538746009), (9, 0.008260210228022013), (10, 0.004130105114011007), (11, 0.001955787484706956), (12, 0.0015918258736505996), (13, 0.0015918258736505996), (14, 0.008260210228022013), (15, 0.0013108697793088472), (16, 0.0011452524080876978), (17, 0.002080991759437162), (18, 0.004839366251287288), (19, 0.0013108697793088472)]


Looking at the above output, the length of the vector has decreased slightly, as values below a certain threshold are removed. The values themselves are now decimals.

### Latent Semantic Indexing (LSI)

A vector representation with ~6000 dimensions is difficult to use, because there are so many degrees of freedom. Latent Semantic Indexing will compress this representation, while minimizing information loss. There are a lot of mathematical details glossed over, but that's okay because GenSim handles the implementation for us.

In [6]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)
corpus_lsi = lsi_model[corpus_tfidf]
print(len(corpus_lsi[0]))
print(corpus_lsi[0])

15
[(0, 0.18682238167974372), (1, -0.4437583954806601), (2, 0.22275580411969662), (3, 0.06534575527078117), (4, -0.10021080420155845), (5, 0.06653745783577146), (6, 0.05025291839076259), (7, 0.7117552624193217), (8, -0.3768886513901333), (9, 0.1650380936828472), (10, 0.13664364557932132), (11, -0.03947144082104315), (12, -0.03177275640769521), (13, -0.00890543444745628), (14, -0.009715808633565214)]


The vector representation is so much smaller now, making document comparison easier. In this specific example, there are 15 vectors instead of 20 as any vector below a certain threshold is filtered out.

### Evaluating Model Similarity

We can use these vectors to evaluate model similarity. By using some fancy math (cosine similarity), we can take two vectors and output a single number representing how similar the two vectors are (scale of 0 to 1). By creating a matrix with these similarities, we can pick out the three documents each document is most similar to.

In [7]:
index = similarities.MatrixSimilarity(corpus_lsi, num_features=20)
print(index)

MatrixSimilarity<15 docs, 20 features>


In [8]:
categories = ["adventure", "belles_lettres", "editorial", "fiction", "government", 
              "hobbies", "humor", "learned", "lore", "mystery", "news", "religion",
              "reviews", "romance", "science_fiction"]
num_categories = len(categories)


for i in range(3):
    print(categories[i])
    sims = index[lsi_model[corpus_bow[i]]]
    top3 = sorted(enumerate(sims), key=lambda x: x[1], reverse=True,)[1:4]
    for j, score in top3:
        print(score, categories[j])
    print()

adventure
0.22929086 fiction
0.20346783 romance
0.19324714 mystery

belles_lettres
0.3659389 editorial
0.3413822 lore
0.33065677 news

editorial
0.45590898 news
0.38146105 government
0.2897901 belles_lettres



The code looks to have run very well! Adventure texts are most similar to fiction, romance, and mystery. Belles lettres, or literary and artistic critque, is most similar to editorials. Finally editorials are most similar to the news and government. The numeric scores also intuitively make sense.

I hope you enjoyed this tutorial using NLTK and GenSim for document similarity